In [159]:
%matplotlib inline

import pandas as pd
import numpy as np

from pandas.api.types import CategoricalDtype

from collections import defaultdict, Counter

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
import matplotlib.dates as mdates

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

import dataStatsAnalysis as dsa
import dataStatsPlotting as dsp

from functools import reduce

dsp.SetParams()

In [160]:
# Load the existing chi-square function
def ResampleChiSquare(observed, expected, iters=1000):
    """Generates a chisquared statistic sampling distribution by randomly choosing values 
    according to the expected probablities to simulate the null hypothesis. 
    The sequences must be the same length, be integer counts of a categorical variable 
    and the sum of the sequence values must be the same. 
    If the sum of the sequence values is different, first normalize the expected values 
    and then create a new expected values sequence by multiplying by the total number of observed values. 
    adjust_expected = expected/sum(expected)*sum(observed) 
    Can then make an rv of this distribution to plot cdf and  
    compute a p-value for the actual chi-squared statistic (eg. rv.cdf at actual statistic (test_chi)). 
    Can also use the 'min' and 'max' built-ins to find what the most extreme values are from the simluations.

    Args:
        observed (array-like): observed values sequence
        expected (array-like): expected values sequence
        iters (int, optional): [description]. Defaults to 1000.

    Returns:
        test_chi: Original actual chi squared value
        chis (array): Sampling distribution for the null hypothesis obtained from resampling
    """
    observed, expected = np.array(observed), np.array(expected)
    
    # Check that sum of values are euqal
    if np.isclose(sum(observed), sum(expected)) == False:
        raise ValueError('The sum of the values for observed and expected must be equal.')
    
    # Calculate the chi square test statistic
    test_chi = sum((observed - expected)**2 / expected)
        
    # Calculate the variables needed for resampling
    n = sum(expected)
    values = list(range(len(expected)))
    p_exp = expected/sum(expected)
    
    # Build the chi square sampling distribution for the null hypothesis
    chis=[]
    for _ in range(iters):
        # Build a model_observed sequence generated by resampling using expected probabilities
        hist = Counter({x:0 for x in values})
        hist.update(np.random.choice(values, size=n, replace=True, p=p_exp))
        sorted_hist = sorted(hist.items())
        model_observed = np.array([x[1] for x in sorted_hist])

        # Compute chi square statistic and append
        chi = sum((model_observed - expected)**2 / expected)
        chis.append(chi)
    
    return test_chi, np.array(chis)

In [161]:
# Build the contingency chi square function
# Calculating the expected within my function should not be necessary
# The scipy chi square contingency function can do this for me
def ResampleChiSquareContingency_old(observed, iters=1000):
    # Put the data into array form
    observed = np.asarray(observed, dtype=np.float64)
    
    # Calculate the marginal sums
    # From https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/contingency.py
    margsums = []
    ranged = list(range(observed.ndim))
    for k in ranged:
        marg = np.apply_over_axes(np.sum, observed, [j for j in ranged if j != k])
        margsums.append(marg)
    
    # Calculate the expected contingency table
    # From https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/contingency.py
    d = observed.ndim
    expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)

In [162]:
def ResampleChiSquareContingency(observed, iters=1000):
    # Put the data into array form
    observed = np.asarray(observed, dtype=np.float64)
    
    # Calculate the test chi square statistic and the expected array
    test_chi,_,_,expected = stats.chi2_contingency(observed)
    
    # Calculate variables to be used in resampling
    expected = np.asarray(expected, dtype=np.float64)
    expected_shape = expected.shape
    expected_ps = expected / np.sum(expected)
    values = np.array(list(range(len(expected.ravel())))) # Flatten the array and then reshape it later
    n= int(np.sum(expected))
      
    # Compute resampled expected values and compute chi square 
    # to build a sampling distribution that represents the null hypothesis
    chis=[]
    for _ in range(iters):
        hist = Counter({x:0 for x in values}) # Initiate an empty histogram to hold resampled values
        hist.update(np.random.choice(values, size=n, replace=True, p=expected_ps.ravel()))
        sorted_hist = sorted(hist.items())
        resampled_expected = np.array([x[1] for x in sorted_hist])
        resampled_expected_reshaped = resampled_expected.reshape(expected_shape) # Put back into original shape

        chi = stats.chi2_contingency(resampled_expected_reshaped)[0]
        chis.append(chi)

    return test_chi, np.array(chis)

Figure out how to resample a contingency table.<br>
Will first try method #2 suggested here:<br>
https://stats.stackexchange.com/questions/303939/bootstrap-resampling-for-contingency-table

I think I have to use the expected contingency table that is output from the scipy function.<br>
The reason is that I need to use the observed test_chi against the null to find the p-value.<br>
The below code seems to be working just need to change to expected I think.<br>
Changed to expected in the final function and it's now working.

In [163]:
observed = [[10,13,15],
            [13,14,15]]

In [164]:
observed = np.asarray(observed, dtype=np.float64)
observed

array([[10., 13., 15.],
       [13., 14., 15.]])

In [165]:
observed_shape = observed.shape
observed_shape

(2, 3)

In [166]:
observed_ps = observed / np.sum(observed)
observed_ps

array([[0.125 , 0.1625, 0.1875],
       [0.1625, 0.175 , 0.1875]])

In [167]:
values = np.array(list(range(len(observed.ravel()))))
values

array([0, 1, 2, 3, 4, 5])

In [168]:
n= int(np.sum(observed))

In [169]:
hist = Counter({x:0 for x in values})
hist

Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0})

In [170]:
hist.update(np.random.choice(values, size=n, replace=True, p=observed_ps.ravel()))
sorted_hist = sorted(hist.items())
model_observed = np.array([x[1] for x in sorted_hist])
model_observed

array([ 8, 14, 17, 19, 12, 10])

In [171]:
model_observed_reshaped = model_observed.reshape(observed_shape)
model_observed_reshaped

array([[ 8, 14, 17],
       [19, 12, 10]])

In [172]:
test_chi,p,dof,expected = stats.chi2_contingency(observed)
test_chi, p

(0.22891366903571353, 0.8918504347412783)

In [173]:
np.sum(expected)

80.0

Test out the new function

In [198]:
chi_cont_results = ResampleChiSquareContingency(observed)
chi_cont_results

(0.22891366903571353,
 array([2.00475448e+00, 3.48711711e+00, 2.05651491e+00, 4.50515881e-01,
        1.71023169e+00, 1.32197244e-01, 2.27574667e+00, 7.75795698e-01,
        1.06748460e+00, 1.32147688e+00, 1.44300144e+00, 3.77441842e+00,
        2.39458605e+00, 7.38619658e+00, 3.80173550e+00, 2.98012584e+00,
        2.03565873e+00, 1.58740995e+00, 3.77534623e+00, 1.34240363e+00,
        4.02387042e+00, 4.16476330e-01, 2.68077601e-01, 1.08845057e+00,
        1.39885208e+00, 3.02947466e+00, 2.30578588e-01, 3.99062172e+00,
        1.07806743e+00, 2.67745150e+00, 1.33421110e+00, 1.34676434e-01,
        3.92251427e-01, 7.42292277e+00, 2.96703297e-01, 1.20968087e+00,
        1.26502901e+00, 2.08216624e+00, 8.18845032e-01, 1.73186294e+00,
        5.99073441e-01, 7.75097918e-01, 7.42505506e-01, 1.59916274e+00,
        5.58186911e-01, 1.14871795e+00, 1.75901691e-01, 4.29213508e+00,
        5.41042610e+00, 3.33346242e+00, 1.15440115e-01, 5.97732576e-01,
        1.87945436e+00, 4.03846154e-01, 2.

In [199]:
# This is VERY close to the result from the scipy funtion above.
# I'm sure this is accurate
dsa.PvalueFromEstimates(chi_cont_results[1], chi_cont_results[0], tail='right')

0.8899999999999999

In [200]:
values.reshape([2,3])

array([[0, 1, 2],
       [3, 4, 5]])

In [201]:
values

array([0, 1, 2, 3, 4, 5])

In [178]:
margsums = []
ranged = list(range(observed.ndim))
for k in ranged:
    marg = np.apply_over_axes(np.sum, observed, [j for j in ranged if j != k])
    margsums.append(marg)
margsums

[array([[38.],
        [42.]]),
 array([[23., 27., 30.]])]

In [179]:

d = observed.ndim
expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)
expected

array([[10.925, 12.825, 14.25 ],
       [12.075, 14.175, 15.75 ]])

In [180]:
reduce(np.multiply, margsums)

array([[ 874., 1026., 1140.],
       [ 966., 1134., 1260.]])